In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, fixed, FloatSlider, IntSlider

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
def get_dados(indice='portfolio'):
    if indice=='portfolio':
        df = pd.read_csv('dados/retorno_portfolio.csv', parse_dates=['Date'])
        df.fillna(0, inplace=True)
    if indice=='ACWI':    
        df = pd.read_csv('dados/retorno_portfolio.csv', parse_dates=['Date'])
        df.fillna(0, inplace=True)
        df['retorno'] = df['ACWI']
    else:
        url = f'https://query1.finance.yahoo.com/v7/finance/download/{indice}?period1=852076800&period2=1634743473&interval=1d&events=history&includeAdjustedClose=true'
        df = pd.read_csv(url)
        df['Date'] = pd.to_datetime(df['Date'])
        df['retorno'] = df['Close'].pct_change()
        
    df['year_week'] = df['Date'].dt.strftime('%Y-%U')
    df['retorno acumulado'] = df['retorno'] + 1
    df['retorno acumulado'] = df['retorno acumulado'].cumprod()
    
    df_gsrai = pd.read_csv('dados/GSRAII.csv', parse_dates=['Date']).sort_values('Date')

    df = df.merge(df_gsrai, on='Date')
    return df[['Date', 'retorno acumulado', 'GSRAII Index', 'year_week']].dropna()

### Indices:
- ^GSPC = *SP500*
- ^BVSP = *Bovespa*
- ACWI
- portfolio

In [26]:
# Trocar codigo do indice
indice='^GSPC'
df = get_dados(indice=indice)
df

,Date,retorno acumulado,GSRAII Index,year_week
1,1997-01-03,1.014952,0.13,1997-00
2,1997-01-06,1.014437,0.22,1997-01
3,1997-01-07,1.022008,0.21,1997-01
4,1997-01-08,1.015468,0.22,1997-01
5,1997-01-09,1.024206,0.18,1997-01
...,...,...,...,...
6160,2021-08-19,5.977938,-0.11,2021-33
6161,2021-08-20,6.026607,-0.09,2021-33
6162,2021-08-23,6.077977,-0.02,2021-34
6163,2021-08-24,6.087068,0.08,2021-34


In [18]:
#%% Semanal
df_gp = df.groupby('year_week').agg('last')

In [19]:
for pct in range(1,6):
    df_gp[f'pct_{pct}sem'] = (df_gp['retorno acumulado']/df_gp['retorno acumulado'].shift(pct).values -1) * 100

In [20]:
def classe_gsrai(df, limit=0, media_movel=3):
    
    df=df.copy()
    df['mean'] = df['GSRAII Index'].rolling(media_movel).mean()
    up_down = df['mean']

    df['gsrai_gt_up'] = np.where((df['GSRAII Index'] > up_down) & (df['GSRAII Index'] >= limit), 1, 0)
    df['gsrai_gt_down'] = np.where((df['GSRAII Index'] < up_down) & (df['GSRAII Index'] >= limit), 1, 0)
    df['gsrai_lt_up'] = np.where((df['GSRAII Index'] > up_down) & (df['GSRAII Index'] < limit), 1, 0)
    df['gsrai_lt_down'] = np.where((df['GSRAII Index'] < up_down) & (df['GSRAII Index'] < limit), 1, 0)

    regras = ['gsrai_gt_up', 'gsrai_gt_down', 'gsrai_lt_up', 'gsrai_lt_down']
    df['classe'] = np.nan
    for col in regras:
        df.loc[df[col]==1, 'classe'] = col
    return df.dropna()

In [21]:
df_gp =  classe_gsrai(df=df_gp, limit=0, media_movel=3)

In [22]:
pct =  'pct_3sem'
regras = ['gsrai_gt_up', 'gsrai_gt_down', 'gsrai_lt_up', 'gsrai_lt_down']

In [23]:
lista = []
for col in df_gp['classe'].unique():
    desc = df_gp[df_gp[col]==1][[pct]].describe().rename(columns={pct: col})
    lista.append(desc)
pd.concat(lista, axis=1)

,gsrai_gt_down,gsrai_gt_up,gsrai_lt_down,gsrai_lt_up
count,260.000000,377.000000,340.000000,313.000000
mean,0.731362,1.613290,-2.195612,1.863994
std,2.913398,2.499989,4.546833,4.560710
min,-8.351541,-7.144465,-28.353571,-22.478099
25%,-0.953529,0.237997,-4.049180,-0.309974
50%,0.830090,1.572469,-1.832571,2.010218
75%,2.463103,2.991507,0.526419,4.116662
max,10.210797,12.324682,7.693614,21.037614


In [27]:
@interact(df=fixed(df_gp), 
          pct=['pct_1sem', 'pct_2sem', 'pct_3sem', 'pct_4sem', 'pct_5sem'],
          limit=FloatSlider(min=-1, max=1, step=0.05, value=0),
          media_movel=IntSlider(min=2, max=6, step=1, value=3)
         )
def hist_classes(df, pct, limit=0, media_movel=3):
    df = classe_gsrai(df=df, limit=limit, media_movel=media_movel)
    fig = px.histogram(df, x=pct, color="classe", marginal="box", nbins=100, 
#                        range_y=[0,50],
                       range_x=[df[pct].min()-1,df[pct].max()+1],
                       title=f'Distribuição {indice} {pct} ',
                      )
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.65)
    fig.add_vline(x=0,line_dash="dash")
    
    for i, classe in enumerate(df['classe'].unique()):
        fig.add_annotation(text=f'Total {classe}: {len(df[df["classe"]==classe])}',
                           showarrow=False, yshift=70-(i*19), xshift=450)
    
    return fig

    
hist_classes(df_gp, pct, limit=-1, media_movel=3);

interactive(children=(Dropdown(description='pct', options=('pct_1sem', 'pct_2sem', 'pct_3sem', 'pct_4sem', 'pc…

In [12]:
cut = df_gp[pct].quantile(0.05)
df_gp[(df_gp[pct]<=cut)].sum()

retorno acumulado                                           289.802746
GSRAII Index                                                    -69.86
pct_1sem                                                   -388.174745
pct_2sem                                                    -747.01848
pct_3sem                                                  -1067.847647
pct_4sem                                                  -1105.903804
pct_5sem                                                  -1144.518671
mean                                                        -58.483333
gsrai_gt_up                                                          1
gsrai_gt_down                                                        6
gsrai_lt_up                                                         15
gsrai_lt_down                                                       43
classe               gsrai_lt_downgsrai_lt_downgsrai_lt_downgsrai_l...
dtype: object

In [13]:
df_gp[(df_gp[pct]>cut) & (df_gp['gsrai_lt_down']==1)][pct].describe()

count    299.000000
mean      -1.001708
std        5.365030
min       -9.986539
25%       -5.012577
50%       -2.086641
75%        2.069678
max       21.058606
Name: pct_3sem, dtype: float64

In [14]:
df_gp['2020-01':'2020-18']

,Date,retorno acumulado,GSRAII Index,pct_1sem,pct_2sem,pct_3sem,pct_4sem,pct_5sem,mean,gsrai_gt_up,gsrai_gt_down,gsrai_lt_up,gsrai_lt_down,classe
year_week,,,,,,,,,,,,,,
2020-01,2020-01-10,16.605995,0.52,-1.872446,-0.397537,-0.884720,0.331825,2.610048,0.450000,1,0,0,0,gsrai_gt_up
2020-02,2020-01-17,17.033714,0.48,2.575691,0.655016,2.167914,1.668183,2.916062,0.453333,1,0,0,0,gsrai_gt_up
2020-03,2020-01-24,17.019050,0.17,-0.086092,2.487381,0.568360,2.079956,1.580655,0.390000,0,1,0,0,gsrai_gt_down
2020-04,2020-01-31,16.355546,-0.36,-3.898594,-3.981330,-1.508186,-3.352392,-1.899728,0.096667,0,0,0,1,gsrai_lt_down
2020-05,2020-02-07,16.356840,0.08,0.007911,-3.890991,-3.973734,-1.500394,-3.344746,-0.036667,1,0,0,0,gsrai_gt_up
2020-06,2020-02-14,16.444684,0.23,0.537048,0.545002,-3.374839,-3.458026,-0.971403,-0.016667,1,0,0,0,gsrai_gt_up
2020-07,2020-02-21,16.344044,-0.15,-0.611990,-0.078228,-0.070323,-3.966176,-4.048853,0.053333,0,0,0,1,gsrai_lt_down
2020-08,2020-02-28,14.976925,-1.84,-8.364634,-8.925433,-8.436319,-8.429075,-11.999054,-0.586667,0,0,0,1,gsrai_lt_down
2020-09,2020-03-06,14.089138,-2.40,-5.927697,-13.796501,-14.324057,-13.863936,-13.857122,-1.463333,0,0,0,1,gsrai_lt_down


In [15]:
df_gp['2019-35':'2019-45']

,Date,retorno acumulado,GSRAII Index,pct_1sem,pct_2sem,pct_3sem,pct_4sem,pct_5sem,mean,gsrai_gt_up,gsrai_gt_down,gsrai_lt_up,gsrai_lt_down,classe
year_week,,,,,,,,,,,,,,
2019-35,2019-09-06,14.799080,-0.56,1.779799,5.393838,3.135082,-1.020232,0.254203,-0.840000,0,0,1,0,gsrai_lt_up
2019-36,2019-09-13,14.880454,-0.10,0.549862,2.339447,5.973358,3.702182,-0.475980,-0.533333,0,0,1,0,gsrai_lt_up
2019-37,2019-09-20,15.069657,-0.25,1.271485,1.828338,3.640678,7.320794,5.020740,-0.303333,0,0,1,0,gsrai_lt_up
2019-38,2019-09-27,15.107181,-0.51,0.249005,1.523657,2.081896,3.898749,7.588029,-0.286667,0,0,0,1,gsrai_lt_down
2019-39,2019-10-04,14.743872,-0.75,-2.404880,-2.161863,-0.917866,-0.373051,1.400109,-0.503333,0,0,0,1,gsrai_lt_down
2019-40,2019-10-11,14.928043,-0.30,1.249135,-1.185786,-0.939733,0.319804,0.871424,-0.520000,0,0,1,0,gsrai_lt_up
2019-41,2019-10-18,15.057005,-0.16,0.863896,2.123821,-0.332134,-0.083956,1.186462,-0.403333,0,0,1,0,gsrai_lt_up
2019-42,2019-10-25,15.435842,0.05,2.516018,3.401649,4.693275,2.175527,2.429949,-0.136667,1,0,0,0,gsrai_gt_up
2019-43,2019-11-01,15.555460,0.05,0.774934,3.310449,4.202943,5.504578,2.967320,-0.020000,1,0,0,0,gsrai_gt_up


In [16]:
df_gp[(df_gp['pct_ret']<0) & (df_gp['gsrai_lt_down']==1)]['pct_ret'].describe()

KeyError: 'pct_ret'

In [ ]:
df_gp[(df_gp['gsrai_lt_down']==1)]['pct_ret'].describe()

In [ ]:
df_gp[(df_gp['gsrai_lt_down']==1)]['pct_ret_shift'].describe()

In [ ]:
df['retorno acumulado'].plot()

In [ ]:
#%% Grafico

fig = px.line(df, x="Date", y='retorno acumulado')
# mask = df_gp['gsrai_lt_down']==1
# fig = px.line(df_gp, x="Date", y=['GSRAII Index', 'pct_ret'])

# fig.add_trace(go.Scatter(
#     x=df_gp[mask]['Date'],
#     y=df_gp[mask][pct],
#     marker_size=10, mode='markers', name='Queda',
#     ))

fig.show()